# Applying a Different Augmentation Strategy Per Class

Let's say we wished to augment the MNIST dataset, but you wished to use a generator to supply a neural network with data. 

Ordinarily you could write a pipeline that would augment all the data, regardless of the class. However with MNIST you might want to have different pipelines for each of the 10 different classes. 

For example, it would make sense to flip images for the figure 8 both horizontally and vertically and still end up with feasible data. The figure 3 could be flipped vertically but not horizontally. Conversely, the figure 4 should not be flipped either horizontally or vertically. 

We can do this by creating 10 different pipelines, and adding or removing the appropriate operations from each pipeline as required.

Augmentor does not support this natively, but it can be performed easily enough, and here we will learn how. 

First we import the required libraries:

In [1]:
import Augmentor
import numpy as np
import os
import glob
import random
import collections
from PIL import Image

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow import keras

from sklearn import metrics

from matplotlib import pyplot as plt

In [2]:
# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.
np.random.seed(123)

# The below is necessary for starting core Python generated random numbers
# in a well-defined state.
random.seed(123)

# The below set_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see:
# https://www.tensorflow.org/api_docs/python/tf/random/set_seed
tf.random.set_seed(1234)

## 1. Point to a Root Directory

Your root directory must contain subdirectories, one for each class in your machine learning classification problem:

In [3]:
root_directory = 'data/Train/*'

# root_directory = "/home/marcus/Documents/mnist/train/*"

## 2. Scan for folders in the root directory

We use `glob.glob()` to scan for all files in the `root_directory` and only choose those that are directories. These will be out classes:

In [4]:
folders = []
for f in glob.glob(root_directory):
    if os.path.isdir(f):
        folders.append(os.path.abspath(f))

print("Folders (classes) found: %s " % [os.path.split(x)[1] for x in folders])

Folders (classes) found: ['Scab_Apple', 'Normal_Apple', 'Blotch_Apple', 'Rot_Apple'] 


## 3. Create a pipeline for each class

Now we create a pipeline object for each of the classes. MNIST consists of 10 digits, and each digit represents one class:

In [5]:
pipelines = {}
for folder in folders:
    print("Folder %s:" % (folder))
    pipelines[os.path.split(folder)[1]] = (Augmentor.Pipeline(folder))
    print("\n----------------------------\n")

Folder /Users/macbookpro/MakeAiWork2/projects/apple_disease_classification/data/Train/Scab_Apple:
Initialised with 85 image(s) found.
Output directory set to /Users/macbookpro/MakeAiWork2/projects/apple_disease_classification/data/Train/Scab_Apple/output.
----------------------------

Folder /Users/macbookpro/MakeAiWork2/projects/apple_disease_classification/data/Train/Normal_Apple:
Initialised with 67 image(s) found.
Output directory set to /Users/macbookpro/MakeAiWork2/projects/apple_disease_classification/data/Train/Normal_Apple/output.
----------------------------

Folder /Users/macbookpro/MakeAiWork2/projects/apple_disease_classification/data/Train/Blotch_Apple:
Initialised with 116 image(s) found.
Output directory set to /Users/macbookpro/MakeAiWork2/projects/apple_disease_classification/data/Train/Blotch_Apple/output.
----------------------------

Folder /Users/macbookpro/MakeAiWork2/projects/apple_disease_classification/data/Train/Rot_Apple:
Initialised with 114 image(s) found.

We can summarise what was scanned:

In [6]:
for p in pipelines.values():
    print("Class %s has %s samples." % (p.augmentor_images[0].class_label, len(p.augmentor_images)))

Class Scab_Apple has 85 samples.
Class Normal_Apple has 67 samples.
Class Blotch_Apple has 116 samples.
Class Rot_Apple has 114 samples.


## 4. Add operations to the pipelines

Here we will add operations to each of the pipelines. Some operations will be applied to all pipelines, others only to some pipelines.

Here we add a rotate operation to all pipelines (and hence will be applied to all digits):

In [7]:
# for pipeline in pipelines.values():
#     pipeline.set_save_format(save_format="png")
#     pipeline.resize(probability=1, height=100, width=100) #resize all for speed
#     pipeline.flip_left_right(probability=0.3) #article flip
#     pipeline.rotate(probability=0.3, max_left_rotation=0.2, max_right_rotation=0.2) # in addition to flip
#     pipeline.zoom(probability=0.7, min_factor=1.1, max_factor=1.2) #article scale
#     pipeline.crop_centre(probability=0.3, percentage_area=0.9) #article crop
#     pipeline.random_color(probability=0.5, min_factor=0.4, max_factor=0.9) #artice color
#     pipeline.random_contrast(probability=0.5, min_factor=0.9, max_factor=1.4) #article color
#     pipeline.sample(1000)
    




Processing <PIL.Image.Image image mode=RGB size=90x90 at 0x7F8028BADAF0>: 100%|██████████| 1000/1000 [00:03<00:00, 280.21 Samples/s] 
Processing <PIL.Image.Image image mode=RGB size=90x90 at 0x7F806DBB9B20>: 100%|██████████| 1000/1000 [00:02<00:00, 358.73 Samples/s]  
Processing <PIL.Image.Image image mode=RGB size=100x100 at 0x7F8028B93490>: 100%|██████████| 1000/1000 [00:04<00:00, 248.45 Samples/s]
Processing <PIL.Image.Image image mode=RGB size=90x90 at 0x7F80598F97C0>: 100%|██████████| 1000/1000 [00:02<00:00, 355.17 Samples/s]  


Here we add some operations that we only want to apply to certain classes. The figure 8 can be flipped horizontally and vertically:

In [8]:
# pipelines["Normal_Apple"].flip_left_right(probability=0.5)

## 5. Define a class label / class integer map

The classes will have string labels associated with them, depending on the name of each class's parent folder. Here you must map the names of each of your classes with the 0-based index (which must correspond to the test data of your dataset).

In the case of MNIST this is quite easy, the samples for the digit 0 were stored in a folder 0 and have the text label 0, and so on:

In [9]:
integer_labels = {'Blotch_Apple': 0, 
                  'Normal_Apple': 1, 
                  'Rot_Apple': 2, 
                  'Scab_Apple': 3}

## 6. Define pipeline containers to store the pipelines and additional information

Later we will need each pipeline's 0-based integer label as well as its categorical label (depending on the type of neural network you define):

In [10]:
PipelineContainer = collections.namedtuple('PipelineContainer', 
                                           'label label_integer label_categorical pipeline generator')

pipeline_containers = []

for label, pipeline in pipelines.items():
    label_categorical = np.zeros(len(pipelines), dtype=int)
    label_categorical[integer_labels[label]] = 1
    pipeline_containers.append(PipelineContainer(label, 
                                                 integer_labels[label], 
                                                 label_categorical, 
                                                 pipeline, 
                                                 pipeline.keras_generator(batch_size=1)))

## 7. Define a generator function

Neural networks in Keras can be supplied with a generator to supply training data. Because we have one generator for each pipeline, we need to create "generator of generators":

In [11]:
def multi_generator(pipeline_containers, batch_size):
    while True:
        X = []
        y = []
        for i in range(batch_size):
            pipeline_container = random.choice(pipeline_containers)
            image, _ = next(pipeline_container.generator)
            image = image.reshape((100, 100, 3)) # Or (1, 28, 28) for channels_first, see Keras' docs.
            X.append(image)
            y.append(pipeline_container.label_categorical)  # Or label_integer if required by network
        X = np.asarray(X)
        y = np.asarray(y)
        yield X, y

## 8. Create the generator object

Create a generator, `g` to pass data randomly from each pipeline (and hence each class) to a neural network:

In [12]:
batch_size = 32

g = multi_generator(pipeline_containers=pipeline_containers, 
                    batch_size=batch_size)  # Here the batch size can be set to any value

In [14]:
(np.shape(X)[0])

NameError: name 'X' is not defined

32 x 100 x 100 x 3

To generate a batch of 32 images (np.shape 32,100,100,3) and labels (np.shape y =32), at random from a random pipeline defined above, we can use the `next()` function:

In [15]:
X, y = next(g)


ValueError: cannot reshape array of size 24300 into shape (100,100,3)

We can confirm that we are receiving images in batches of 128 and that the labels correspond to the images in each pipeline:

In [ ]:
print (np.shape(X))
print (len(y))

In [ ]:
print (type(X[4]))

print (y[4])

We can use PIL to view the augmented images and cofirm the labels match (note that PIL requires images to be specified differently to how Keras expects data, hence some preprocessing of the data must be performed):

In [ ]:
image_index = 8  # Take image index 3 from the batch

x_array = X[image_index]
# x_array = x_array.reshape((100,100))
x_array = x_array * 255
x_array = x_array.astype(np.uint8)
Image.fromarray(x_array)

The label below should correspond to the image output above:

In [ ]:
print("Image label: %s" % (np.nonzero(y[image_index])[0][0]))

## 9. Train a neural network with the generator

Last, we train a neural network with the differing pipelines for each class.

First we define a neural network:

In [ ]:
num_classes = 4
# lossFunction = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# gdAlgorithm = keras.optimizers.Adam(learning_rate=0.001)
# nrOfEpochs = 5
img_height = 100
img_width = 100
image_size=(img_height, img_width)
batch_size = 32

model = tf.keras.Sequential([
  keras.layers.Conv2D(32, 3,input_shape=(img_height, img_width, 3), activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Conv2D(32, 3, activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Conv2D(32, 3, activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Flatten(),
  keras.layers.Dense(128, activation='relu'),
  keras.layers.Dropout(0.3),
  keras.layers.Dense ((num_classes),activation='softmax')
])



Once a network has been defined, you can compile it so that the model is ready to be trained with data:

In [ ]:
model.compile(optimizer='Adadelta',
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
            metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Display the model's architecture
model.summary()

In [ ]:
# model.compile(loss=keras.losses.categorical_crossentropy,
#               optimizer=keras.optimizers.Adadelta(),
#               metrics=['accuracy'])



Using the same batch size as the generator above, we can begin to train the neural network: 1st try = 382 sample / 32 batch size = 12/epoch

In [ ]:
h = model.fit_generator(g, steps_per_epoch=382/batch_size, epochs=10, verbose=1)

fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
print(hist.history.keys())

In [ ]:
fig = plt.figure()
plt.plot(hist.history['sparse_categorical_accuracy'], color='teal', label='sparse_categorical_accuracy')
plt.plot(hist.history['val_sparse_categorical_accuracy'], color='orange', label='val_sparse_categorical_accuracy')
fig.suptitle('sparse_categorical_accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

## Evaluation

In [ ]:
for element in test_data.as_numpy_iterator(): 
    X, y = element
    yhat = model.predict(X)
    pred = (np.argmax(yhat, axis=-1))
    # pre.update_state(y, yhat)
    # re.update_state(y, yhat)
    # acc.update_state(y, yhat)

    # print (yhat)
    # print (y)

print (y)

In [ ]:

# plt.plot(history.history['accuracy'], label='Train accuracy')
# plt.plot(history_test.history['accuracy'], label = 'Test accuracy')

actual = y
predicted = pred

confusion_matrix = metrics.confusion_matrix(actual, predicted)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = ['Blotch', 'Normal', 'Rot', 'Scab'])

cm_display.plot()
plt.show()

In [ ]:
testscore = model.evaluate(test_data)

In [ ]:
print(pre.result(), re.result(), acc.result())